# Import librairies

In [14]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np

# Import data

In [2]:
path = "../data"
df = pd.read_csv(os.path.join(path,"cleaned_dataset.csv"),index_col=0)

In [3]:
df

,yearsofexperience,yearsatcompany,basesalary,company_AMD,company_Accenture,company_Adobe,company_Airbnb,company_Amazon,company_American Express,company_Apple,...,tag_truncated_others,country_Canada,country_Germany,country_US,country_United Kingdom,year_2017,year_2018,year_2019,year_2020,year_2021
0,1.5,1.5,107000.0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
2,8.0,0.0,155000.0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,7.0,5.0,157000.0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
6,15.0,11.0,180000.0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
7,4.0,4.0,135000.0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,6.0,4.0,147500.0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
35320,4.0,2.0,142000.0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
35321,10.0,1.0,155000.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
35322,2.0,2.0,146900.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


# Add gaussian noise

In [7]:
X = df.drop(columns=["basesalary"])
y = df["basesalary"]

In [20]:
noises_std={}
for col in df.columns:
    noises_std[col] = df[col].std()    

In [21]:
df2 = df.copy()

In [23]:
for col in df2.columns:
    df2[col]=df2[col]+np.random.normal(0,noises_std[col]/3)

,yearsofexperience,yearsatcompany,basesalary,company_AMD,company_Accenture,company_Adobe,company_Airbnb,company_Amazon,company_American Express,company_Apple,...,tag_truncated_others,country_Canada,country_Germany,country_US,country_United Kingdom,year_2017,year_2018,year_2019,year_2020,year_2021
0,0.547385,1.98489,100512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,1.01509,0.085253,-0.040576,-0.217204,-0.061625
2,7.047385,0.48489,148512.614559,-0.044501,-0.001528,-0.081372,-0.015165,1.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,1.01509,0.085253,-0.040576,-0.217204,-0.061625
3,6.047385,5.48489,150512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,1.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,1.01509,0.085253,-0.040576,-0.217204,-0.061625
6,14.047385,11.48489,173512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,1.01509,0.085253,-0.040576,-0.217204,-0.061625
7,3.047385,4.48489,128512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,1.01509,0.085253,-0.040576,-0.217204,-0.061625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35319,5.047385,4.48489,141012.614559,-0.044501,-0.001528,-0.081372,-0.015165,1.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,0.01509,1.085253,-0.040576,-0.217204,-0.061625
35320,3.047385,2.48489,135512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,0.871048,-0.007677,0.013244,0.984805,-0.051821,0.01509,1.085253,-0.040576,-0.217204,-0.061625
35321,9.047385,1.48489,148512.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,0.01509,1.085253,-0.040576,-0.217204,-0.061625
35322,1.047385,2.48489,140412.614559,-0.044501,-0.001528,-0.081372,-0.015165,0.080622,0.011705,0.108017,...,-0.128952,-0.007677,0.013244,0.984805,-0.051821,0.01509,1.085253,-0.040576,-0.217204,-0.061625
